___
Title: Udacity Data Wrangling Project

Author: Simon Thornewill von Essen

Date: 
___

# Import Libraries and Data

## Libaries

In [1]:
import pandas as pd # Handling Dataframes
import numpy as np # Mathematical Python
import tweepy # Twitter API
import requests # Downloading Files Programatically
import sqlalchemy # Exporting to Database
import pickle # Serialisation
import json # Handling JSON files
import os # Handling directories 
from tqdm import * # Progress tracker

# Function Definition

In [2]:
def str_frac_div(x):
    """
    Takes a string x in the format "y/z" where y and z are whole numbers and splits the 
    string into two varibles and divided by each other.
    
    Returns the fraction as a float.
    """
    x_split = x.split("/")
    x_float = int(x_split[0]) / int(x_split[1])
    return x_float

## Data

### Import Manually Downloaded Data

In [3]:
# Import Manually Downloaded Data
df_info = pd.read_csv("data/twitter-archive-enhanced.csv")

df_info.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

### Import Programmatically Downloaded Data

In [4]:
# Import Data Programmatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

# Save Data
with open('data/image-predictions.tsv', "wb") as file: file.write(response.content)
    
# Import Data
df_breeds = pd.read_csv('data/image-predictions.tsv', sep="\t")

df_breeds.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

### Import Data from Twitter API

In [5]:
# Import API Details into Working Memory
df_api = pd.read_csv("data/api-details.csv")

# Set up Twitter API Details
consumer_key = df_api.iloc[0,1]
consumer_secret = df_api.iloc[1,1]
access_token = df_api.iloc[2,1]
access_secret = df_api.iloc[3,1]

# Create Connection to API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Create list of twitter ids
tweet_ids = df_info["tweet_id"].astype(list)

In [6]:
# Write JSON to text file
failed_ids=[]

if not os.path.exists('data/tweet_json.txt'):
    with open('data/tweet_json.txt', 'a', encoding='UTF-8') as file:
        for tweet_id in tqdm(tweet_ids):
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')._json
                json.dump(tweet, file)
                file.write('\n')
            except:
                print(tweet_id, "encountered an error, writing to memory.")
                failed_ids.append(tweet_ids)

In [7]:
# Pickle Errors for later investigation
pickle.dump(failed_ids, open("data/failed_ids.pkl", "wb"))


In [8]:
# Read text file line by line to create dataframe
tweets_data = []
with open('data/tweet_json.txt') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
df_api = pd.DataFrame(tweets_data, columns=list(tweets_data[0].keys()))

df_api.head()

created_at                  id              id_str  \
0  Tue Aug 01 16:23:56 +0000 2017  892420643555336193  892420643555336193   
1  Tue Aug 01 00:17:27 +0000 2017  892177421306343426  892177421306343426   
2  Mon Jul 31 00:18:03 +0000 2017  891815181378084864  891815181378084864   
3  Sun Jul 30 15:58:51 +0000 2017  891689557279858688  891689557279858688   
4  Sat Jul 29 16:00:24 +0000 2017  891327558926688256  891327558926688256   

                                           full_text  truncated  \
0  This is Phineas. He's a mystical boy. Only eve...      False   
1  This is Tilly. She's just checking pup on you....      False   
2  This is Archie. He is a rare Norwegian Pouncin...      False   
3  This is Darla. She commenced a snooze mid meal...      False   
4  This is Franklin. He would like you to stop ca...      False   

  display_text_range                                           entities  \
0            [0, 85]  {'hashtags': [], 'symbols': [], 'user_mentions...   
1           [0, 138]  {'hashtags': [], 'symbols': [], 'user_mentions...   
2           [0, 121]  {'hashtags': [], 'symbols': [], 'user_mentions...   
3            [0, 79]  {'hashtags': [], 'symbols': [], 'user_mentions...   
4           [0, 138]  {'hashtags': [{'text': 'BarkWeek', 'indices': ...   

                                   extended_entities  \
0  {'media': [{'id': 892420639486877696, 'id_str'...   
1  {'media': [{'id': 892177413194625024, 'id_str'...   
2  {'media': [{'id': 891815175371796480, 'id_str'...   
3  {'media': [{'id': 891689552724799489, 'id_str'...   
4  {'media': [{'id': 891327551943041024, 'id_str'...   

                                              source  in_reply_to_status_id  \
0  <a href="http://twitter.com/download/iphone" r...                    NaN   
1  <a href="http://twitter.com/download/iphone" r...                    NaN   
2  <a href="http://twitter.com/download/iphone" r...                    NaN   
3  <a href="http://twitter.com/download/iphone" r...                    NaN   
4  <a href="http://twitter.com/download/iphone" r...                    NaN   

   ...  place  contributors is_quote_status retweet_count favorite_count  \
0  ...   None          None           False          8704          39157   
1  ...   None          None           False          6387          33532   
2  ...   None          None           False          4244          25271   
3  ...   None          None           False          8803          42509   
4  ...   None          None           False          9587          40671   

  favorited retweeted possibly_sensitive possibly_sensitive_appealable  lang  
0     False     False              False                         False    en  
1     False     False              False                         False    en  
2     False     False              False                         False    en  
3     False     False              False                         False    en  
4     False     False              False                         False    en  

[5 rows x 27 columns]

In [9]:
df_api.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'extended_entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive',
       'possibly_sensitive_appealable', 'lang'],
      dtype='object')

In [10]:
# Reduce df_api to the necessary columns
df_response = df_api[['id', 'retweet_count', 'favorite_count']]

df_response.head()

id  retweet_count  favorite_count
0  892420643555336193           8704           39157
1  892177421306343426           6387           33532
2  891815181378084864           4244           25271
3  891689557279858688           8803           42509
4  891327558926688256           9587           40671

In [11]:
# Save Dataframe for Visual Assessment
df_response.to_csv("data/tweet_json.csv")

# Copy Dataframes for Cleaning
df_info_clean = df_info.copy()
df_breeds_clean = df_breeds.copy()
df_response_clean = df_response.copy()

# Data Assessment

## Visual Assessment

Visual assessment was carried out by opening csv files in Calc (Libre Office)

Quality Issues:
* Some ids did not work using the twitter API
* Some tweets within df_breeds do not contain dogs. In order to assertain which rows to remove, a sample of pictures needed to be assesed visually. However, a programmatic approach was used in order to find populations from which the samples could be withdrawn from. [Link](http://localhost:8888/notebooks/wrangling-act.ipynb#df_breeds)
* These columns in df_info_clean contain non-useful information:
    * "source", "expanded_urls"
* Dog names and stages in df_info_clean also do not always map properly to their respective columns

Tidiness Issues:
* df_info contains information about various tweets which is similar in theme to the data within df_recieved.


## Programatic Assessment

### df_info

In [12]:
# Check Data Types of Each Dataframe
df_info_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Comments:
* There are a number of missing values within this dataframe:
    * in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, 

Explanation of Attributes:

Quality Issues: 
* A number of posts are retweets or are in response to other comments. These comments often do not follow the standard procedure of posting.
* tweet_id is a categorical value but is labeled as an integer (ordinal value)
* the numerators and demominators to not always map correctly to the score
* text column contains the shortened URL to the tweet
* Although it looks as if the dog state is complete, there are strings set to "None", which shows that these columns are far more empty than would otherwise be indicated.
    * What's even worse is that when all of these "None" values are removed, there are very few categorisations left over. Only about 90/2000 of categorisations can be made.
    * Hence, these are just useless columns. 


Tidiness Issues:
* doggo, floofer, pupper and puppo columns that have a common "theme" of being the stage of the dog within the tweet, they should be collapsed into one column named dog_type.
    * Note: After doing so it was found that the information density of this column was very low.




___

### df_breeds

In [13]:
df_breeds_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Comments:
* There are no missing entries in this dataframe
* Most columns has an appropriate datatype
* Although the CNN does not recognise the dog within the picture right away, it sometimes recognises it as the 2nd or 3rd highest probability. 

Explanation of attributes:
* tweet_id: id of tweet
* jpg_url: web address of dog image
* img_num: a column of ones
* p1, p2, p3: Prediction of breed in descending order of probability
* p1_conf, p2_conf, p3,conf: Probability of prediction in descending order of probability
* p1_dog, p2_dog, p3_dog: Whether or not p1 $\rightarrow$ p3 is a breed of dog

Quality Issues:
* "img_num" is provding no new information
* p1_dog $\rightarrow$ p3_dog indicate pictures where either no dog is present or one is present but wasn't recognised
* tweet_id is a categorical value but is labeled as an integer (ordinal value)

In [14]:
# Check if CNN was at least able to tell what isn't a dog
df_breeds[(df_breeds["p1_dog"] == False) &
         (df_breeds["p2_dog"] == False) &
         (df_breeds["p3_dog"] == False)]

tweet_id                                            jpg_url  \
6     666051853826850816    https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
17    666104133288665088    https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg   
18    666268910803644416    https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg   
21    666293911632134144    https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg   
25    666362758909284353    https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg   
29    666411507551481857    https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg   
45    666786068205871104    https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg   
50    666837028449972224    https://pbs.twimg.com/media/CUEUva1WsAA2jPb.jpg   
51    666983947667116034    https://pbs.twimg.com/media/CUGaXDhW4AY9JUH.jpg   
53    667012601033924608    https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg   
56    667065535570550784    https://pbs.twimg.com/media/CUHkkJpXIAA2w3n.jpg   
69    667188689915760640    https://pbs.twimg.com/media/CUJUk2iWUAAVtOv.jpg   
73    667369227918143488    https://pbs.twimg.com/media/CUL4xR9UkAEdlJ6.jpg   
77    667437278097252352    https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg   
78    667443425659232256    https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg   
93    667549055577362432    https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg   
94    667550882905632768    https://pbs.twimg.com/media/CUObvUJVEAAnYPF.jpg   
96    667724302356258817    https://pbs.twimg.com/media/CUQ7tv3W4AA3KlI.jpg   
98    667766675769573376    https://pbs.twimg.com/media/CURiQMnUAAAPT2M.jpg   
100   667782464991965184    https://pbs.twimg.com/media/CURwm3cUkAARcO6.jpg   
106   667866724293877760    https://pbs.twimg.com/media/CUS9PlUWwAANeAD.jpg   
107   667873844930215936    https://pbs.twimg.com/media/CUTDtyGXIAARxus.jpg   
112   667911425562669056    https://pbs.twimg.com/media/CUTl5m1WUAAabZG.jpg   
115   667937095915278337    https://pbs.twimg.com/media/CUT9PuQWwAABQv7.jpg   
117   668142349051129856    https://pbs.twimg.com/media/CUW37BzWsAAlJlN.jpg   
118   668154635664932864    https://pbs.twimg.com/media/CUXDGR2WcAAUQKz.jpg   
123   668226093875376128    https://pbs.twimg.com/media/CUYEFlQXAAUkPGm.jpg   
130   668291999406125056    https://pbs.twimg.com/media/CUZABzGW4AE5F0k.jpg   
132   668466899341221888    https://pbs.twimg.com/media/CUbfGbbWoAApZth.jpg   
140   668544745690562560    https://pbs.twimg.com/media/CUcl5jeWsAA6ufS.jpg   
...                  ...                                                ...   
1839  837482249356513284    https://pbs.twimg.com/media/C59VqMUXEAAzldG.jpg   
1844  838916489579200512    https://pbs.twimg.com/media/C6RkiQZUsAAM4R4.jpg   
1847  839290600511926273    https://pbs.twimg.com/media/C6XBt9XXEAEEW9U.jpg   
1851  840370681858686976    https://pbs.twimg.com/media/C6mYrK0UwAANhep.jpg   
1853  840696689258311684    https://pbs.twimg.com/media/C6rBLenU0AAr8MN.jpg   
1869  844580511645339650    https://pbs.twimg.com/media/C7iNfq1W0AAcbsR.jpg   
1886  847962785489326080    https://pbs.twimg.com/media/C8SRpHNUIAARB3j.jpg   
1887  847971574464610304    https://pbs.twimg.com/media/C8SZH1EWAAAIRRF.jpg   
1891  849051919805034497    https://pbs.twimg.com/media/C8hwNxbXYAAwyVG.jpg   
1892  849336543269576704    https://pbs.twimg.com/media/C8lzFC4XcAAQxB4.jpg   
1900  851464819735769094    https://pbs.twimg.com/media/C9ECujZXsAAPCSM.jpg   
1902  851861385021730816    https://pbs.twimg.com/media/C8W6sY_W0AEmttW.jpg   
1905  852226086759018497  https://pbs.twimg.com/ext_tw_video_thumb/85222...   
1906  852311364735569921    https://pbs.twimg.com/media/C9QEqZ7XYAIR7fS.jpg   
1910  853299958564483072    https://pbs.twimg.com/media/C9eHyF7XgAAOxPM.jpg   
1931  859074603037188101    https://pbs.twimg.com/media/C-wLyufW0AA546I.jpg   
1936  860184849394610176    https://pbs.twimg.com/media/C-_9jWWUwAAnwkd.jpg   
1937  860276583193509888    https://pbs.twimg.com/media/C_BQ_NlVwAAgYGD.jpg   
1940  860924035999428608    https://pbs.twimg.com/media/C_KVJjDXsA

It seems that the CNN is relatively good at detecting what is *not* a dog. Hence, it would be good to remove rows for which p1_dog $\rightarrow$ p3_dog is False. 

In [15]:
# Check how accurate the CNN is when the first selection is a dog
df_breeds[(df_breeds["p1_dog"] == True)]

tweet_id                                            jpg_url  \
0     666020888022790149    https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1     666029285002620928    https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2     666033412701032449    https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3     666044226329800704    https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4     666049248165822465    https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5     666050758794694657    https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
7     666055525042405380    https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
9     666058600524156928    https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10    666063827256086533    https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   
11    666071193221509120    https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg   
12    666073100786774016    https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg   
13    666082916733198337    https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg   
14    666094000022159362    https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg   
15    666099513787052032    https://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg   
16    666102155909144576    https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg   
19    666273097616637952    https://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg   
20    666287406224695296    https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg   
23    666345417576210432    https://pbs.twimg.com/media/CT9Vn7PWoAA_ZCM.jpg   
24    666353288456101888    https://pbs.twimg.com/media/CT9cx0tUEAAhNN_.jpg   
26    666373753744588802    https://pbs.twimg.com/media/CT9vZEYWUAAlZ05.jpg   
27    666396247373291520    https://pbs.twimg.com/media/CT-D2ZHWIAA3gK1.jpg   
28    666407126856765440    https://pbs.twimg.com/media/CT-NvwmW4AAugGZ.jpg   
30    666418789513326592    https://pbs.twimg.com/media/CT-YWb7U8AA7QnN.jpg   
31    666421158376562688    https://pbs.twimg.com/media/CT-aggCXAAIMfT3.jpg   
32    666428276349472768    https://pbs.twimg.com/media/CT-g-0DUwAEQdSn.jpg   
34    666435652385423360    https://pbs.twimg.com/media/CT-nsTQWEAEkyDn.jpg   
35    666437273139982337    https://pbs.twimg.com/media/CT-pKmRWIAAxUWj.jpg   
36    666447344410484738    https://pbs.twimg.com/media/CT-yU5QWwAEjLX5.jpg   
37    666454714377183233    https://pbs.twimg.com/media/CT-5Bs-WUAA2JeC.jpg   
38    666644823164719104    https://pbs.twimg.com/media/CUBl6IwVAAA9_zT.jpg   
...                  ...                                                ...   
2041  885311592912609280    https://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg   
2042  885528943205470208    https://pbs.twimg.com/media/DEoH3yvXgAAzQtS.jpg   
2043  885984800019947520    https://pbs.twimg.com/media/DEumeWWV0AA-Z61.jpg   
2044  886258384151887873    https://pbs.twimg.com/media/DEyfTG4UMAE4aE9.jpg   
2045  886366144734445568    https://pbs.twimg.com/media/DE0BTnQUwAApKEH.jpg   
2047  886736880519319552    https://pbs.twimg.com/media/DE5Se8FXcAAJFx4.jpg   
2048  886983233522544640    https://pbs.twimg.com/media/DE8yicJW0AAAvBJ.jpg   
2049  887101392804085760    https://pbs.twimg.com/media/DE-eAq6UwAA-jaE.jpg   
2050  887343217045368832  https://pbs.twimg.com/ext_tw_video_thumb/88734...   
2051  887473957103951883    https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg   
2053  887705289381826560    https://pbs.twimg.com/media/DFHDQBbXgAEqY7t.jpg   
2054  888078434458587136    https://pbs.twimg.com/media/DFMWn56WsAAkA7B.jpg   
2055  888202515573088257    https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg   
2056  888554962724278272    https://pbs.twimg.com/media/DFTH_O-UQAACu20.jpg   
2057  888804989199671297    https://pbs.twimg.com/media/DFWra-3VYAA2piG.jpg   
2058  888917238123831296    https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg   
2059  889278841981685760  https://pbs.twimg.com/ext_tw_video_thumb/88927...   
2060  889531135344209921    https://pbs.twimg.com/media/DFg_2PVW0AEHN3p.jpg   
2061  889638837579907072    https://pbs.twimg.com/media/DFihzFfXsA

I also want to keep rows that contain the first value is True or all values are True, the CNN will be pretty confident that these images are of dogs. 

In [16]:
# Checking selection when the 2nd or 3rd probabilities are dogs
df_breeds[((df_breeds["p1_dog"] == False) &
         ((df_breeds["p2_dog"] == True) |
         (df_breeds["p3_dog"] == True)))]

tweet_id                                            jpg_url  \
8     666057090499244032    https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
22    666337882303524864    https://pbs.twimg.com/media/CT9OwFIWEAMuRje.jpg   
33    666430724426358785    https://pbs.twimg.com/media/CT-jNYqW4AAPi2M.jpg   
43    666776908487630848    https://pbs.twimg.com/media/CUDeDoWUYAAD-EM.jpg   
52    666996132027977728    https://pbs.twimg.com/media/CUGlb6iUwAITEbW.jpg   
87    667524857454854144    https://pbs.twimg.com/media/CUOGUfJW4AA_eni.jpg   
95    667550904950915073    https://pbs.twimg.com/media/CUOb_gUUkAACXdS.jpg   
103   667806454573760512    https://pbs.twimg.com/media/CUSGbXeVAAAgztZ.jpg   
108   667878741721415682    https://pbs.twimg.com/media/CUTILFiWcAE8Rle.jpg   
113   667915453470232577    https://pbs.twimg.com/media/CUTpj-GWcAATc6A.jpg   
126   668256321989451776    https://pbs.twimg.com/media/CUYflCXWEAAzQVu.jpg   
131   668297328638447616    https://pbs.twimg.com/media/CUZE4IWW4AAZmDf.jpg   
133   668480044826800133    https://pbs.twimg.com/media/CUbrDWOWcAEyMdM.jpg   
145   668625577880875008    https://pbs.twimg.com/media/CUdvambWoAA007z.jpg   
150   668641109086707712    https://pbs.twimg.com/media/CUd9ivxWUAAuXSQ.jpg   
158   668872652652679168    https://pbs.twimg.com/media/CUhQIAhXAAA2j7u.jpg   
163   668960084974809088    https://pbs.twimg.com/media/CUifpn4WUAAS5X3.jpg   
167   668986018524233728    https://pbs.twimg.com/media/CUi3PIrWoAAPvPT.jpg   
171   668994913074286592    https://pbs.twimg.com/media/CUi_UtnWIAEtfqz.jpg   
179   669324657376567296    https://pbs.twimg.com/media/CUnrN7vUcAAfGvN.jpg   
183   669353438988365824    https://pbs.twimg.com/media/CUoFZTyW4AE70iD.jpg   
198   669625907762618368    https://pbs.twimg.com/media/CUr9NjgU8AEpf5w.jpg   
210   669993076832759809    https://pbs.twimg.com/media/CUxLJO8U8AAu6Zu.jpg   
213   670040295598354432    https://pbs.twimg.com/media/CUx2F6lVEAAvFev.jpg   
235   670427002554466305    https://pbs.twimg.com/media/CU3VzVwWwAAAsst.jpg   
237   670433248821026816    https://pbs.twimg.com/media/CU3be0SWEAEqb7I.jpg   
238   670434127938719744    https://pbs.twimg.com/media/CU3cSG8W4AIAePH.jpg   
239   670435821946826752    https://pbs.twimg.com/media/CU3d0azWUAA38FD.jpg   
243   670452855871037440    https://pbs.twimg.com/media/CU3tUC4WEAAoZby.jpg   
261   670786190031921152    https://pbs.twimg.com/media/CU8ceuxWUAALMEo.jpg   
...                  ...                                                ...   
1748  823581115634085888    https://pbs.twimg.com/media/C23ypm6VQAAO31l.jpg   
1761  826115272272650244    https://pbs.twimg.com/media/C3bzVILWcAUjS5i.jpg   
1779  828409743546925057    https://pbs.twimg.com/media/C38aQYgXAAMY2Wh.jpg   
1781  828708714936930305    https://pbs.twimg.com/media/C4AqLSgVYAEg8nt.jpg   
1782  828770345708580865    https://pbs.twimg.com/media/C4BiOXOXAAAf6IS.jpg   
1817  834086379323871233    https://pbs.twimg.com/media/C5NFIsjWQAEI93t.jpg   
1821  834574053763584002    https://pbs.twimg.com/media/C5UAqgyXAAAbMWH.jpg   
1823  834931633769889797    https://pbs.twimg.com/media/C5ZF4p-XEAEmApg.jpg   
1824  835152434251116546    https://pbs.twimg.com/media/C5cOtWVWMAEjO5p.jpg   
1826  835264098648616962    https://pbs.twimg.com/media/C5d0QtvXMAI_7uz.jpg   
1833  836753516572119041    https://pbs.twimg.com/media/C5y-4VwWcAIcaoj.jpg   
1834  836989968035819520    https://pbs.twimg.com/media/C52V7PzWcAA_pVv.jpg   
1848  839549326359670784    https://pbs.twimg.com/media/C6atpTLWYAIL7bU.jpg   
1856  841439858740625411    https://pbs.twimg.com/media/C61lFFiWoAAJdiL.jpg   
1858  841833993020538882  https://pbs.twimg.com/ext_tw_video_thumb/81742...   
1862  842765311967449089    https://pbs.twimg.com/media/C7IalMVX0AATKRD.jpg   
1872  844979544864018432    https://pbs.twimg.com/media/C7n4aQ0VAAAohkL.jpg   
1881  847116187444137987    https://pbs.twimg.com/media/C8GPrNDW4AAkLde.jpg   
1896  850145622816686080    https://pbs.twimg.com/media/C8xS655XkA

When the top probability is not a dog then it seems that there is usually a dog present in the picture then the first match had a probability score of less than 0.5

If the picture is recognised to have a dog with more than a 0.25 rating, then there is a decent probability that it has a dog. 

This will be my selection criteria for making sure I have only categorisations that have a high probability of having a dog inside of the picture
___
### df_clean

In [17]:
df_response_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 3 columns):
id                2384 non-null int64
retweet_count     2384 non-null int64
favorite_count    2384 non-null int64
dtypes: int64(3)
memory usage: 56.0 KB


Comments:
* No missing data from datafame

Explanation of attributes:
* id: equivalent to "tweet_id" in other dataframes
* retweet_count: number of times tweet has been retweeted
* favorite_count: number of times tweet has bene favorited

Quality Issues:
* id is a numerical value when it should be a string

Quality Issue: 
* tweet_id is incorrectly labeled as ID

# Data Cleaning

## Summary of Assessment

Quality: 
1. Some ids did not work using the twitter API, this means that tweet_ids might not be consistent across dataframes
2. "img_num" in df_breeds is not providing any new info
3. tweets in df_breeds do not always contain dogs and when they do they are sometimes not categorised correctly
4. "tweet_id" is incorrectly labeled as "id" in df_response_clean
5. "tweet_id" in df_info_clean and df_breeds_clean as well as "id" in df_response_clean are integers when they should be strings
6. "source" and "expanded_urls" in df_info_clean are not useful
7. Numerators and demominators indf_info_clean to not always map correctly to the score
8. Dog names and stages also do not always map properly to their respective columns, even when you do, only 90 or so can be categorised programatically 
9. text column contains the shortened URL to the tweet
10. Although it looks as if the dog state is complete, there are strings set to "None", which shows that these columns are far more empty than would otherwise be indicated.
11. A number of posts in df_info_clean are retweets or are in response to other comments. These comments often do not follow the standard procedure of posting.

Tidiness:
1. "doggo", "floofer", "pupper" and "puppo" columns in df_info_clean that have a common "theme" of being the stage of the dog within the tweet, and contain very little information what would be difficult to fill out.
2. Info from df_response_clean and df_info_clean both contain information about tweets

## Define Steps

type since it is not contained within the text and there is no programmatic way to assess this.
1. Remove shortened URL from the end of the "text" column in the df_info_clean dataframe
2. Remove rows that are retweets or responses and delete their respective columns
3. Remove useless columns from df_info_clean (including dog stage)
4. Remove column "img_num" from df_breeds_clean
5. Remove rows in df_breeds_clean where:
    * All probabilities of p_dog are False
    * The Probabilities of p2_dog and p2_dog are lower than 0.2
6. Rename "id" to "tweet_id" in df_response_clean
7. Convert "tweet_id" in df_info_clean and df_breeds_clean as well as "id" in df_response_clean from integers to strings 
8. Make sure that IDs are consistent across dataframes by removing rows whose ID does not appear in the other two dataframes:
    * Removed ids from df_breeds should be removed from other dataframes
    * Failed ids that didnt make it into df_recieved should also be removed from other dataframes
9. Use regular expressions to find the numerator and denominator of the score and place save them in the correct column ("numerator", "denominator")
10. Check dog names in df_info_clean using regular expressions, do not double check the dog 
11. Since p1_dog is always true and has the highest probability of being the correct 
prediction, I will remove p1_dog and p1_conf, change p1 to "predicted_breed" and change
the "_" within the strings to " " and make all characters lowercase.

*N.B. These steps are within the under that they should be undertaken, not in the same order as they appeared in the summary assessment*

___

## Carry out Steps

In [18]:
"""
1. Remove shortened URL from the end of the "text" column in the df_info_clean dataframe
"""
# Find the legnth of URLs
length_of_url = len(df_info_clean["text"][0].split()[-1])+1
length_of_url

# Remove the length of URLs from text entries
df_info_clean["text"] = df_info_clean["text"].apply(lambda row: row[:-length_of_url])

In [19]:
"""
2. Remove rows that are retweets or responses and delete their respective columns
"""
# Convert from integer to string
df_info_clean["in_reply_to_status_id"] = df_info_clean["in_reply_to_status_id"].astype(str)
df_info_clean["retweeted_status_id"] = df_info_clean["retweeted_status_id"].astype(str)

# Find ids to remove
replies = df_info_clean[df_info_clean["in_reply_to_status_id"]!="nan"]["tweet_id"].index
df_info_clean.drop(replies, inplace=True)

retweets = df_info_clean[df_info_clean["retweeted_status_id"]!="nan"]["tweet_id"].index
df_info_clean.drop(retweets, inplace=True)


# Replace weird "nan" strings with actual NaNs
df_info_clean["in_reply_to_status_id"].replace("nan", np.nan, inplace=True)
df_info_clean["retweeted_status_id"].replace("nan", np.nan, inplace=True)

In [20]:
"""
3. Remove useless columns from df_info_clean
"""
remove_cols_info = ["source", "expanded_urls", "doggo", "floofer", "pupper", "puppo",
                   "in_reply_to_status_id", "in_reply_to_user_id", 
                   "retweeted_status_id", "retweeted_status_user_id", 
                    "retweeted_status_timestamp"]
df_info_clean.drop(columns=remove_cols_info, inplace=True)

In [21]:
"""
4. Remove column "img_num" from df_breeds_clean
"""
df_breeds_clean.drop(columns=("img_num"), inplace=True)

In [22]:
"""
5. Remove rows in df_breeds_clean where:
    * All probabilities of p_dog are False
    * The Probabilities of p2_conf and p2_dog are lower than 0.2
"""
# Find indexes of pictures not of dogs
not_dog = df_breeds_clean[(df_breeds_clean["p1_dog"] == False) &
         (df_breeds_clean["p2_dog"] == False) &
         (df_breeds_clean["p3_dog"] == False)]["tweet_id"].index

# Find indexes of pictures that are unlikely to be dogs
unlikely_dog = df_breeds_clean[(df_breeds_clean["p1_dog"] == False) &
         (df_breeds_clean["p2_conf"] < 0.2) &
         (df_breeds_clean["p3_conf"] < 0.2)]["tweet_id"].index

# Put both series into the same variable
remove_row = not_dog.append(unlikely_dog)

df_breeds_clean.drop(remove_row, inplace=True)

In [23]:
"""
6. Rename "id" to "tweet_id" in df_response_clean
"""
df_response_clean.rename({"id": "tweet_id"}, axis=1, inplace=True)

In [24]:
"""
7. Convert "tweet_id" in df_info_clean and df_breeds_clean as well as "id" in 
df_response_clean from integers to strings
"""
df_info_clean["tweet_id"] = df_info_clean["tweet_id"].astype("str")
df_breeds_clean["tweet_id"] = df_response_clean["tweet_id"].astype("str")
df_response_clean["tweet_id"] = df_response_clean["tweet_id"].astype("str")

In [25]:
"""
8. Make sure that IDs are consistent across dataframes by removing rows whose ID does not 
appear in the other two dataframes:
    * Removed ids from df_breeds should be removed from other dataframes
    * Failed ids that didnt make it into df_recieved should also be removed from other 
    dataframes
    * Do this by merging files, keeping track of IDs was too cumbersome
"""
# Merge info with response (name info)
df_temp = df_info_clean.merge(df_response_clean, how="inner", on="tweet_id").copy()

# Remove Duplicates
df_temp.drop_duplicates(inplace=True)

# Merge breeds with info
df_clean = df_temp.merge(df_breeds_clean, how="inner", on="tweet_id").copy()

# Turns out that the same text can have multiple tweet_ids
# Drop duplicates according to "text" column. 
df_clean.drop_duplicates(subset='text', keep="first", inplace=True)

In [26]:
"""
9. Use regular expressions to find the numerator and denominator of the score and place 
save them in the correct column ("numerator", "denominator")
# """
# String match for correct format of ratings
df_clean_frac_format = df_clean.text.str.contains("\d+/10")

# Remove rows that do not match format
df_clean = df_clean[df_clean_frac_format].copy()

# Extract numerators and denominators
df_clean_frac = df_clean.text.str.extract("(\d+/10)", expand=False).copy()

df_clean["rating_numerator"] = df_clean_frac.apply(lambda x: int(str(x)[:-3]))
df_clean["rating_denominator"] = 10


In [27]:
"""
10. Check dog names in df_info_clean using regular expressions, do not double check the dog 
"""
# Find string that contain format "This is <name>." and remove rows that do not have it.
df_clean_name_format = df_clean["text"].str.contains("This is ([A-Z][a-z]+).")

# Extract names and add to proper column
df_clean = df_clean[df_clean_name_format].copy()

# Extract names
df_clean["name"] = df_clean.text.str.extract("This is ([A-Z][a-z]+).", expand=False)

/home/simon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [28]:
"""
11. Since p1_dog is always true and has the highest probability of being the correct 
prediction, I will remove p1_dog and p1_conf, change p1 to "predicted_breed" and change
the "_" within the strings to " " and make all characters lowercase.
"""
# Remove all "pN_dog" and "pN_conf" columns
df_clean.drop(columns=["p1_dog", "p1_conf", 
                       "p2_dog", "p2_conf", 
                       "p3_dog", "p3_conf", 
                       "p2", "p3"], inplace=True)

# Change p1 to "predicted_breed"
df_clean.rename(columns={"p1": "predicted_breed"}, inplace=True)

# Change all alphabetical characters in "predicted_breed" to lower-case
# change all "_" to " " within strings
df_clean["predicted_breed"] = df_clean["predicted_breed"].apply(lambda x: 
                                                                x.lower().replace("_", " "))


___

## Test Changes 

In [29]:
"""
1. Remove shortened URL from the end of the "text" column in the df_info_clean dataframe
"""

if len(df_info.text[0]) - len(df_info_clean.text[0]) == length_of_url:
    print("URL was successfully deleted")

URL was successfully deleted


In [30]:
"""
2. Remove rows that are retweets or responses and delete their respective columns
"""
try: 
    df_info_clean[unlikely_dog]
except KeyError: print("Columns successfully deleted")

Columns successfully deleted


In [31]:
"""
3. Remove useless columns from df_info_clean
"""
try: 
    df_info_clean[remove_cols_info]
except KeyError: print("Columns successfully deleted")

Columns successfully deleted


In [32]:
"""
4. Remove column "img_num" from df_breeds_clean
"""
try: 
    df_breeds_clean["img_num"]
except KeyError: print("Columns successfully deleted")

Columns successfully deleted


In [33]:
"""
5. Remove rows in df_breeds_clean where:
    * All probabilities of p_dog are False
    * The Probabilities of p2_dog and p2_dog are lower than 0.2
"""
# Check certain not dogs
assert len(df_breeds_clean[(df_breeds_clean["p1_dog"] == False) &
     (df_breeds_clean["p2_dog"] == False) &
     (df_breeds_clean["p3_dog"] == False)]) == 0
print("Not dogs successfully deleted")

# Check probably not dogs
assert len(df_breeds_clean[(df_breeds_clean["p1_dog"] == False) &
                    (df_breeds_clean["p2_conf"] < 0.2) &
                    (df_breeds_clean["p3_conf"] < 0.2)]) == 0
print("Unlikely dogs successfully deleted")

Not dogs successfully deleted
Unlikely dogs successfully deleted


In [34]:
"""
6. Rename "id" to "tweet_id" in df_response_clean
"""
if df_response_clean.columns[0] == "tweet_id": print("Columns successfully renamed")

Columns successfully renamed


In [35]:
"""
7. Convert "tweet_id" in df_info_clean and df_breeds_clean as well as "id" in 
df_response_clean from integers to strings
"""
if df_info_clean["tweet_id"].dtype == 'O': 
    print("df_info_clean Dtype change successful")
if df_breeds_clean["tweet_id"].dtype == 'O': 
    print("df_breeds_clean Dtype change successful")
if df_response_clean["tweet_id"].dtype == 'O': 
    print("df_response_clean Dtype change successful")

df_info_clean Dtype change successful
df_breeds_clean Dtype change successful
df_response_clean Dtype change successful


In [36]:
"""
8. Make sure that IDs are consistent across dataframes by removing rows whose ID does not 
appear in the other two dataframes:
    * Removed ids from df_breeds should be removed from other dataframes
    * Failed ids that didnt make it into df_recieved should also be removed from other 
    dataframes
    * Do this by merging files, keeping track of IDs was too cumbersome
"""
if not df_clean.duplicated().sum(): print("Merge was successful")

Merge was successful


In [37]:
"""
9. Use regular expressions to find the numerator and denominator of the score and place 
save them in the correct column ("numerator", "denominator")
"""

# calculate values from extraction
df_frac = df_clean.text.str.extract("(\d+/10)", expand=False).apply(lambda x: str_frac_div(x))
df_frac2 = df_clean["rating_numerator"]/df_clean["rating_denominator"]

# Assert that the calculation worked
assert df_frac.equals(df_frac2)
print("Calculation and saving to corect columns successful")

Calculation and saving to corect columns successful


In [38]:
"""
10. Check dog names in df_info_clean using regular expressions, do not double check the dog 
type since it is not contained within the text and there is no programmatic way to assess 
this.
"""
df_clean.name

0        Phineas
2          Tilly
5         Archie
7          Darla
10      Franklin
19          Zoey
20        Cassie
23          Koda
25         Bruno
31           Ted
33        Stuart
34        Oliver
38           Jim
40          Zeke
42       Ralphus
44       Jeffrey
46        Canela
49          Maya
50        Mingus
51         Derek
52        Roscoe
53       Waffles
54        Maisey
56          Earl
57          Lola
58         Kevin
60          Noah
61         Bella
64         Rusty
65           Gus
          ...   
1342    Chuckles
1343        Chet
1344      Gustaf
1345       Terry
1347       Oscar
1348          Ed
1349       Jerry
1350    Leonidas
1352      Norman
1354         Taz
1357      Jackie
1360       Buddy
1361       Franq
1362      Pippin
1364        Kreg
1366        Rolf
1369      Ridley
1371         Cal
1372        Opal
1373     Bradley
1374       Bubba
1376        Tuco
1377       Patch
1379        Lola
1380      Batdog
1381        Brad
1382         Mia
1390       Osc

In [39]:
"""
11. Since p1_dog is always true and has the highest probability of being the correct 
prediction, I will remove p1_dog and p1_conf, change p1 to "predicted_breed" and change
the "_" within the strings to " " and make all characters lowercase.
"""
df_clean["predicted_breed"] 

0            welsh springer spaniel
2                           redbone
5                   german shepherd
7               rhodesian ridgeback
10               miniature pinscher
19                        chihuahua
20                 miniature poodle
23                 golden retriever
25                    gordon setter
31                              pug
33                       bloodhound
34                            lhasa
38                   english setter
40           curly-coated retriever
42                        dalmatian
44                        chihuahua
46                         pembroke
49                 golden retriever
50               miniature pinscher
51                  giant schnauzer
52                       toy poodle
53                           vizsla
54                       rottweiler
56                         papillon
57                    saint bernard
58                       rottweiler
60                  tibetan terrier
61                       toy

# Export Data

## Export to CSV

In [44]:
# Write to CSV file
df_clean.to_csv('data/wrangled/wr_rate_dogs.csv')

## Export to SQLite Database

In [43]:
# Create engine
engine = sqlalchemy.create_engine('sqlite:///data/wrangled/we_rate_dogs.db')

# Create database
df_clean.to_sql('master', engine, index=False)